*** DA3 OPEN STREET MAP - DATA WRANGLING WITH MONGODB***

*Map Area: Berlin, Germany*

https://www.openstreetmap.org/#map=13/52.5180/13.4076

The area contains the central area of Berlin. As it is one of the biggest cities in Europe, we can expect a lot of additional information about this area from the osm project.

In [ ]:
#Sample of nodes in an .osm file:

<node id="302864488" visible="true" version="7" changeset="36059354" timestamp="2015-12-20T06:35:42Z" user="atpl_pilot" uid="881429" lat="52.5259586" lon="13.3894424">
  <tag k="addr:city" v="Berlin"/>
  <tag k="addr:country" v="DE"/>
  <tag k="addr:housenumber" v="45"/>
  <tag k="addr:postcode" v="10117"/>
  <tag k="addr:street" v="Oranienburger Straße"/>
  <tag k="addr:suburb" v="Mitte"/>
  <tag k="amenity" v="restaurant"/>
  <tag k="contact:phone" v="+493028040505"/>
  <tag k="cuisine" v="cuban"/>
  <tag k="name" v="QBA"/>
  <tag k="website" v="http://www.qba-restaurant.de/"/>
  <tag k="wheelchair" v="no"/>
 </node>

<node id="270497666" visible="true" version="3" changeset="21935727" timestamp="2014-04-25T16:17:56Z" user="HolgerJeromin" uid="67862" lat="52.5279489" lon="13.3837052">
  <tag k="cemetery" v="grave"/>
  <tag k="image" v="http://commons.wikimedia.org/wiki/Image:Tombstone_Karl_Friedrich_Schinkel.jpg"/>
  <tag k="name" v="Karl Friedrich Schinkel"/>
</node >
 

In [7]:
#1. Size of File
import xml.etree.ElementTree as ET
import pprint
import re
import os
import codecs
import json
from collections import defaultdict

#https://www.openstreetmap.org/export#map=13/52.5180/13.4076
OSM_FILE = 'berlin_map.osm'
SAMPLE_FILE = 'berlin_map_reduced.osm'

In [8]:
#check file size
#resource: http://stackoverflow.com/questions/2104080/how-to-check-file-size-in-python

def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0


def file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)
    
size = file_size(OSM_FILE)
print ('OSMSize', size)

OSMSize 154.6 MB


**Create sample file with the k - th size**

In [9]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

k = 14 # Parameter: take every k-th top level element
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag
    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
    context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()

with open(SAMPLE_FILE, 'wb') as output:
    b = bytearray()
    b.extend('<?xml version="1.0" encoding="UTF-8"?>\n'.encode())
    b.extend('<osm>\n  '.encode())
    output.write(b)

    # Write every kth top level element
    print (OSM_FILE)
    for i, element in enumerate(get_element(OSM_FILE)):

        if not i % k:
            output.write(ET.tostring(element, encoding='utf-8'))
    b_end = bytearray()
    b_end.extend('</osm>'.encode())
    output.write(b_end)

berlin_map.osm


In [10]:
#check size of sample file
sample_size = file_size(SAMPLE_FILE)
print ('SampleSize', sample_size)

SampleSize 11.3 MB


**AUDITING THE .OSM FILE**

In [11]:
#get benchmark data
"""
    Reference:
    https://classroom.udacity.com/nanodegrees/nd002/parts/0021345404/modules/316820862075462/lessons/768058569/concepts/8426285720923#
"""
def get_benchmark_data(filename):
    users = set()
    count_nodes = 0
    count_ways = 0
    count_relations = 0
    
    for _, element in ET.iterparse(filename):
        if element.tag == 'node':
            count_nodes += 1
            user = element.attrib['uid']
            if user in users:
                pass
            else:
                users.add(user)
        if element.tag == 'way':
            count_ways += 1
        if element.tag == 'relation':
            count_relations += 1
    return users, count_nodes, count_ways, count_relations

users, count_nodes, count_ways, count_relations = get_benchmark_data(OSM_FILE)

print ('UNIQUE USERS: ', len(users))
print ('NODES:', count_nodes)
print ('WAYS:', count_ways)
print ('RELATIONS', count_relations)

UNIQUE USERS:  2003
NODES: 594858
WAYS: 86201
RELATIONS 3203


**AUDIT STREET NAMES**

*Problems encounterd while auditing street names:*

   In German language there a lot of different namings and writings for streets: For example a 'Straße' can be 'Auerstraße' or     'Antwerpener Straße' or 'Alfred-Jung-Straße' or 'Straße des 17. Juni'. This happens also to other street types, like              '*weg*', '*zeile*', '*platz*' and also for streets that are or used to be near rivers like: '*damm*', '*ufer*', '*graben*'.
   Because there a lot of bridges in Berlin, bridges belong to ways *brücke*. 
   A street name starting with 'Zur ', 'Am ' or 'An ' simply means 'at ', but is also a valid street name. 
    
   Another problem is the case sensitiveness. Those key words may be written with uppercase or lowercase. In our case we simply    transform it to lowercase, to audit the street names. 
    
   Another problem are the german Umlaute. Because we don't wan't to get troubled, instead of using regex, we will use a simple    loop to audit the street names.
    
   A further task would be to transform all Umlaute of the file at the beginning, to make sure not to get in trouble later.

In [31]:
'''reference:
https://classroom.udacity.com/nanodegrees/nd002/parts/0021345404/modules/316820862075462/lessons/768058569/concepts/8755386150923#'''

#start with lower case letters, due to different ways of writing street names in german
expected = ['straße', 'platz', 'gasse', 'weg', 'allee', 'damm', 'ufer', 'graben', 'brücke', 
            'promenade', 'park', 'am', 'an', 'markt', 'steg', 'hof', 'zeile']

def audit_street_type(street_types, street_name):
    found = False
    for e in expected:
        if e in street_name.lower():
            found = True
            break
           
    if found:
        street_types[e].add(street_name)
    else:
        street_types[street_name.title()].add(street_name)    
                              
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

#audit the street names
def audit_street_names(osmfile):
    osm_file = open(osmfile, 'rt', encoding='utf-8')
    #set is a list without duplicates
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    print('Numbers of different street types in Berlin')
    pretty_print(street_types)

def pretty_print(d):
    for sorted_key in sorted(d, key=lambda k: len(d[k]), reverse=True):
        v = d[k]
        print (sorted_key.title(), ':', len(d[sorted_key]))
        
audit_street_names(OSM_FILE)

Numbers of different street types in Berlin
Straße : 620
Platz : 48
Ufer : 28
Allee : 18
Damm : 12
Weg : 9
Am : 8
Park : 6
An : 5
Brücke : 4
Graben : 4
Zeile : 3
Promenade : 3
Markt : 3
Hof : 2
Westring : 1
Alt-Moabit : 1
Fischerinsel : 1
Prenzlauer Berg : 1
Zur Waage : 1
Stadtbahnbogen : 1
Wriezener Karree : 1
In Den Ministergärten : 1
Steg : 1
Südstern : 1
Dohnagestell : 1
Südring : 1
Hinter Der Katholischen Kirche : 1
Zur Innung : 1
Fischzug : 1
Großer Stern : 1
Zur Börse : 1
Unter Den Linden : 1
Neue Welt : 1
Südpassage : 1
Vor Dem Schlesischen Tor : 1
Alt-Stralau : 1
Viehtrift : 1


It seems that all street names are valid!

**Audit Post codes and suburbs**
The postal code in Berlin ranges from 10115 to 14199 
In the following task we want to check if the postal code is valid for Berlin and if it matches with the suburb according to thist list : https://en.wikipedia.org/wiki/List_of_postal_codes_in_Germany#Berlin

10115, 10117, 10119, 10178, 10179 - Berlin-Mitte
10243, 10245, 10247, 10249 - Friedrichshain
10318, 10319 - Lichtenberg
10405, 10407, 10409, 10435, 10437, 10439 - Prenzlauer Berg
10551, 10553, 10555, 10557, 10559 - Tiergarten
10585, 10587, 10589, 10623, 10625, 10627, 10629 - Charlottenburg
10707, 10709, 10711, 10713, 10715, 10719 - Wilmersdorf
10777, 10779, 10781, 10783, 10789 - Tempelhof
10823, 10825, 10827, 10829 - Schöneberg
10961, 10963, 10965, 10967, 10969, 10997, 10999 - Kreuzberg
12043, 12045, 12047, 12049, 12051, 12053, 12055, 12057, 12059 - Neukölln
12157, 12161, 12163, 12165, 12167, 12169 - Steglitz
12203, 12205, 12207, 12209 - Lichterfelde
12247, 12249 - Lankwitz
12277, 12279 - Marienfelde
12305, 12307, 12309 - Lichtenrade
12487, 12489 - Adlershof
12555, 12557, 12559, 12587 - Köpenick
12679, 12681, 12683, 12685, 12687, 12689 - Marzahn since 1979
13086, 13088, 13089 - Weißensee
13187, 13189 - Pankow
13347, 13349, 13351, 13353, 13355, 13357, 13359 - Wedding
13403, 13405, 13407, 13409 - Reinickendorf
13435, 13437, 13439 - Wittenau
14109 - Wannsee
14163, 14165, 14167, 14169 - Zehlendorf

In [12]:
#TODO AUDIT Postal codes and suburbs
10115 to 14199 

expected_suburbs = ['Berlin-Mitte', 'Friedrichshain', 'Lichtenberg', 'Prenzlauer Berg', 'Tiergarten',
                   'Charlottenburg', 'Wilmersdorf', 'Tempelhof', 'Schöneberg', 'Kreuzberg', 'Neukölln', 
                   'Steglitz', 'Lichterfelde', 'Lankwitz', 'Marienfelde', 'Lichtenrade',  'Adlershof',
                   'Köpenick', 'Marzahn', 'Weißensee', 'Pankow','Wedding', 'Reinickendorf', 'Wittenau',
                  'Wannsee', 'Zehlendorf']
def audit_suburbs(expected_suburbs, suburb):
    #TODO


#check if postal code is correct and then check if it corresponds to correct suburbs
def audit_postal_code(postal_range, postal_code):
#TODO
#TODO Check if postal code corresponds to correct suburb

def is_postal_code(elem):
    return (elem.attrib['k'] == "addr:postcode")

def is_suburb(elem):
    return (elem.attrib['k'] == "addr:surbub")

#check if the postal_codes are correct
#check if the postal code belongs to the right suburb
def audit_postalcodes(osmfile):
    osm_file = open(osmfile, 'rt', encoding='utf-8')
    #set is a list without duplicates
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_code(postal_codes, tag.attrib['v'])
    osm_file.close()
    print('Postal Codes')
    pretty_print(postal_codes)



SyntaxError: invalid syntax (<ipython-input-12-0ea96855774e>, line 2)

In [33]:
### 2. transform to json
''' 
reference:
https://classroom.udacity.com/nanodegrees/nd002/parts/0021345404/modules/316820862075462/lessons/768058569/concepts/8755386150923#'''

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')
addr = re.compile(r'^([a-z]|_)*$')

def shape_element(element):
    node = {}
    created_dict = {}
    pos_list = []
    node_refs_list = []
    long = 0
    lat = 0

    if element.tag == "node" or element.tag == "way" :
        
        node["type"] = element.tag
        for name, value in element.items():
            if name in CREATED:
                created_dict[name] = value
            elif name == 'long':
                lon = float(value)
            elif name == 'lat':
                lat = float(value)
            else:
                node[name] = value

            if len(created_dict):
                node["created"] = created_dict
            if lat:
                pos_list.append(lat)
            if long:
                pos_list.append(long)
            if len(pos_list):
                node["pos"] = pos_list

        address = {}
        for tag in element.iter("tag"):
            k = tag.attrib['k']
            v = tag.attrib['v']
            if lower.search(k):
                node[k] = v
            elif lower_colon.search(k):
                match = addr.search(k)

                if match:
                    if street_name(k):
                        better_name = update_name(v, mapping)
                        address[match.groups()[0]] = better_name
                    else:
                        address[match.groups()[0]] = v
                elif problemchars.search(k):
                    pass

            if len(address):
                node['address'] = address

        for nd in element.iter("nd"):
            node_refs_list.append(nd.attrib['ref'])
            if len(node_refs_list):
                node["node_refs"] = node_refs_list
                return node
            else:
                return None

#fix typos before exporting to mongodb
mapping = { "Str.": "Straße",
            "Strase": "Straße",
            "Strasse": "Straße"}
       
def update_name(name, mapping):
    m = street_type_re.search(name)
    if m.group() in mapping.keys():
        match = re.search(street_type_re,name)
        name = re.sub(street_type_re,mapping[match.group()],name)
    if m.group() not in mapping.keys():
        for ind in diction1:
            if m.group() in diction1[ind]:
                name = re.sub(m.group(), ind + ' ' + m.group(), name)
    return name            
            
            
def process_map(file_in):
    # You do not need to change this file
    file_out = "{0}.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                fo.write(json.dumps(el) + "\n")
    return data

data = process_map('berlin_map.osm')

**Show one element of the generated .json data**

In [21]:
example = data[5]
print (example)

{'postal_code': '10623', 'highway': 'primary', 'lanes': '3', 'surface': 'asphalt', 'ref': 'B 2;B 5', 'maxspeed': '50', 'created': {'version': '33', 'user': 'jacobbraeutigam', 'uid': '1260280', 'timestamp': '2016-06-12T20:13:17Z', 'changeset': '39978827'}, 'type': 'way', 'oneway': 'yes', 'lit': 'yes', 'name': 'Straße des 17. Juni', 'id': '4068038', 'node_refs': ['21487168']}


**Save data to mongoDB**

In [34]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)

db = client.berlin

def insert_osm_data(infile, db):
    db.berlin.drop()      
    #import data into a collection named "berlin"
    db.berlin.insert_many(infile)
    print (db.berlin.find_one())

insert_osm_data(data, db)

{'_id': ObjectId('587bd779c7993a1088b727ea'), 'type': 'way', 'maxspeed': '8', 'boat': 'yes', 'waterway': 'canal', 'have_riverbank': 'yes', 'id': '4041237', 'name': 'Landwehrkanal', 'draft': '1.40', 'node_refs': ['2087107436'], 'created': {'version': '17', 'user': 'Krille von Stralau', 'uid': '597190', 'timestamp': '2012-12-29T12:55:05Z', 'changeset': '14450125'}}


In [50]:
print (len(db.berlin.distinct("created")));

50789


**Get statistics of our database**

In [51]:
print (db.command("dbstats"))

{'dataSize': 22989032.0, 'storageSize': 4096.0, 'objects': 86201, 'collections': 1, 'numExtents': 0, 'views': 0, 'avgObjSize': 266.69101286527996, 'indexes': 1, 'db': 'berlin', 'ok': 1.0, 'indexSize': 4096.0}


**Count ways**

In [102]:
db.berlin.find({"type":"way"}).count()

86165

**Count unique users**

In [131]:
print (len(db.berlin.distinct("created.user")));

1188


**Get top 10 contributiong users**

In [27]:
def pretty_print_list(d):
    for member in d:
        print (member['_id'], ':', member['count'])    
        
# get top ten of contributing users
def get_top_ten_users():
    pipeline = [{'$group' : { '_id' : '$created.user', 'count' : {'$sum' : 1}}},       
                {'$sort': {'count': -1}},
                { '$limit': 10}]
    return pipeline

def aggregate(db, pipeline):
    return [doc for doc in db.berlin.aggregate(pipeline)]

pipeline = get_top_ten_users()
result = aggregate(db, pipeline)
print ('Top 10 contributing users with counted entries:')
pretty_print_list(result);

Top 10 contributing users with counted entries:
atpl_pilot : 20031
anbr : 6537
MorbZ : 5727
Bot45715 : 3023
Jojo4u : 2605
Shmias : 2319
Polarbear : 1999
haytigran : 1859
geozeisig : 1840
RoterEmil : 1541


**Get an overview of different ways**

In [28]:
def get_overview_of_types():
    pipeline =  [{'$group' : { '_id' : '$type', 'count' : {'$sum' : 1}}},       
                {'$sort': {'count': -1}}]
    return pipeline

pipeline = get_overview_of_types()
result = aggregate(db, pipeline)
print ('Types of ways with counted entries:')
pretty_print_list(result)  

Types of ways with counted entries:
way : 86165
multipolygon : 29
property_line : 4
bazar : 1
noise_barrier : 1
sewage : 1


**Get top 10 amenities**

In [30]:
def get_overview_amenities():
    pipeline = [{'$match':{'amenity':{'$exists':1}}},
                {'$group' : { '_id': '$amenity', 'count' : {'$sum':1}}},
               {'$sort': {'count': -1}},
               {'$limit': 5}]
    return pipeline
    
pipeline = get_overview_amenities()
result = aggregate(db, pipeline)
print ('Amenities:')
pretty_print_list(result)

Amenities:
parking : 837
school : 230
kindergarten : 173
place_of_worship : 119
bicycle_parking : 98


**Get number of bycicle roads**

As we can see, there are 98 bycicle_parkings counted, which seems to be a lot. Because in Berlin there is currently a referndum  - https://volksentscheid-fahrrad.de/english/ - to make the city more bycicle friendly, we would like to take a closer look at the bycicle ways in the city.
Because of the mixture of german and english there are different nodes to mark cicleways. http://wiki.openstreetmap.org/wiki/Key:bicycle_road

In [274]:
def get_roads():
    pipeline = [{ '$group': {'_id': 'highway', 'count': {'$sum': 1}}}]
    return pipeline
    
pipeline = get_roads()
result = aggregate(db, pipeline)
print ('Roads:')
pretty_print_list(result)
print ('~~~~~~~~~~~~~~~~~~~~~~~')       
    
#find bycicle roads in berlin
def get_bicycle_roads():
    pipeline = [{ '$match': {'$or':
                    [{'bicycle': { '$in': ['official', 'designated', 'use_sidepath']}},
                    {'bicycle_road': 'yes'},
                    {'cycleway': {'$in': ['lane', 'opposite', 'shared', 'share_busway', 'track']}}]
                    }},
                {'$group' : { '_id': '$highway', 'count' : {'$sum':1}}},
                {'$sort': {'count': -1}}]          
    return pipeline

pipeline = get_bicycle_roads()
result = aggregate(db, pipeline)
print ('Bicycle Roads:')
pretty_print_list(result)
print ('~~~~~~~~~~~~~~~~~~~~~~~')   

def get_total_amount_of_cycleways():   
    pipeline = [{ '$match': {'$or':
                    [{'bicycle': { '$in': ['official', 'designated', 'use_sidepath']}},
                    {'bicycle_road': 'yes'},
                    {'cycleway': {'$in': ['lane', 'opposite', 'shared', 'share_busway', 'track']}}]
                    }},
                {'$group' : { '_id': None, 'count' : {'$sum':1}}}] 
    return pipeline
    
pipeline = get_total_amount_of_cycleways()
result = aggregate(db, pipeline)
print ('Bicycle Roads Total:')
pretty_print_list(result)
print ('~~~~~~~~~~~~~~~~~~~~~~~')   


Roads:
highway : 86201
~~~~~~~~~~~~~~~~~~~~~~~
Bicycle Roads:
secondary : 996
primary : 529
tertiary : 347
residential : 195
path : 66
service : 20
cycleway : 20
pedestrian : 17
footway : 6
living_street : 6
construction : 3
secondary_link : 2
primary_link : 1
None : 1
~~~~~~~~~~~~~~~~~~~~~~~
Bicycle Roads Total:
None : 2209
~~~~~~~~~~~~~~~~~~~~~~~


Compared to the total number of highways, the number of bicycle roads in this map is extremly small
It would be also interresting to calculate the length of all bicycle ways and compare them to the normal street net.

*Hunger*

Because Berlin is known for its masses of restaurants we want to take a closer look at the types of restaurants:

In [282]:
def get_overview_amenities():
    pipeline = [{'$match': {'amenity': {'$exists': 1}, 
                            'amenity': {'$in': ['restaurant', 'fast_food', 'food_court', 'biergarten', 'bar', 'bbq', 'cafe'] 
                           }}},
               {"$group":{"_id":"$cuisine", "count":{"$sum":1}}},
               {"$sort":{"count":-1}}]

    return pipeline
    
pipeline = get_overview_amenities()
result = aggregate(db, pipeline)
print ('Restaurants by cuisine:')
pretty_print_list(result)

Restaurants by cuisine:
None : 89
burger : 6
regional : 5
kebab : 5
german : 5
asian : 5
donut : 2
pizza : 2
turkish : 2
international : 1
steak_house;fish : 1
sushi : 1
sandwich : 1
american : 1
greek : 1
sausage : 1
different : 1
cuban : 1
ice_cream : 1
coffee_shop : 1
arabian : 1
spanish : 1
vegetarian : 1
ethiopian : 1
regional,_national,_international : 1
italian : 1
Frühstück_Kaffee_&_Kuchen_Eis_Flohmarkt-Gelände : 1
swiss : 1
fish : 1
bavarian : 1
fish_and_chips : 1


**Conclusion**

*problems encounterd*
- The German Umlaute may cause problems, when we are using regex to audit files.
- The German language has a lot of valid declarations for street names, so we have to audit it more often than english street names in order to make shure everything is valid.
- The mixture between English and German words makes it a little bit more difficult to compare values. 
- There are different data-keys for bicycle-ways, so everything is mixed here.
- When counting restaurants, it also shows that there is a mixture of languages. Which is shown with the entries 'coffee_shop' and '*Kaffee&Kuchen*', which is the same.
- There is data missing, because there are far more restaurants in this area, than our query shows. 

*additional ideas*
- Maybe it would be to calculate the length of all bycicley ways compared to the 'normal' street length. 
- It also would be interessting to calculate the the percentage of the surface types, because it is said that Berlin is a "green" city.
- Maybe it would be nice, if there is an easier way to add data. So that more people, like tourists can add data.

In [ ]:
#TODO: 
- create pdf and html file for the analysis
- create stand alone python file

-
Submission document includes one or more additional suggestions for improving and analyzing the data.
One of the requirements of this project is to provide suggestions for improving and analyzing the data. Do not worry if the idea is difficult to implement or if it falls outside the scope of the project. We are just looking for innovative ways to improve the data. Following are some suggestions to get you started:

Sample project has some good ideas: https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md
How about using the popular game Pokemon Go to improve the dataset?
Can we use third party tools like Google Maps API to improve the dataset?


Submission document includes thoughtful discussion about the benefits as well as some anticipated problems in implementing the improvement.
You are also required to include a discussion about the benefits as well as some anticipated problems in implementing the improvement. You can add two subsections as below:

Benefits:

point1
point2

Anticipated Problems:

point1
point2
